In [100]:
import bayeslite
from bayeslite.read_pandas import bayesdb_read_pandas_df
import bdbcontrib
from bdbcontrib import cursor_to_df as df
from bdbcontrib.recipes import quickstart
import pandas as pd
import numpy
import re
import matplotlib
from matplotlib import ft2font
import matplotlib.pyplot as plt
%matplotlib inline
from bdbcontrib.recipes import quickstart
import crosscat
import crosscat.MultiprocessingEngine as ccme
import bayeslite.metamodels.crosscat
import os

In [101]:
import pandas as pd
import crosscat
from bdbcontrib.recipes import quickstart
import bayeslite
from bayeslite.read_pandas import bayesdb_read_pandas_df

In [61]:
DATA = pd.read_csv("Most+Recent+Cohorts+(All+Data+Elements).csv", delimiter=',', low_memory = False)
DATA = DATA.replace('PrivacySuppressed', '-1')
#Restricting the dataframe to 57 variables of interest
df =DATA.loc[:,['OPEID','PFTFAC','UGDS','CCUGPROF','CCBASIC','locale2','region','st_fips','ACTCM75','ACTCMMID','ACTCM25','SAT_AVG_ALL','SATWR75','SATWRMID','SATWR25','SATMT75','SATMTMID','SATMT25','SATVR75','SATVRMID','SATVR25','ADM_RATE_ALL','WOMENONLY','MENONLY','AANAPII','PBI','md_earn_wne_p10','mn_earn_wne_p10','md_faminc','faminc','TUITIONFEE_PROG','AVGFACSAL','PCTPELL','IND_DEBT_MDN','HI_INC_DEBT_MDN','MD_INC_DEBT_MDN','LO_INC_DEBT_MDN','WDRAW_DEBT_MDN','GRAD_DEBT_MDN','MD_INC_RPY_5YR_RT','NOLOAN_COMP_ORIG_YR2','LOAN_COMP_ORIG_YR2','NOPELL_COMP_ORIG_YR2','PELL_COMP_ORIG_YR2','HI_INC_COMP_ORIG_Y','MD_INC_COMP_ORIG_Y','LO_INC_COMP_ORIG_Y','DEATH_YR2_RT','NOLOAN_ENRL_ORIG_YR','NOLOAN_DEATH_YR2_RT','LOAN_DEATH_YR2_RT','NOPELL_DEATH_YR2_RT','PELL_DEATH_YR2_RT','HI_INC_DEATH_YR2_R','MD_INC_DEATH_YR2_R','LO_INC_DEATH_YR2_R','DEP_DEBT_MDN']] 
#Renaming the variables names by their definition
df.columns = ['8-digit OPE ID for institution','Faculty Rate','Enrollment of undergraduate degree-seeking students','Carnegie Classification -- undergraduate profile','Carnegie Classification -- basic','Degree of urbanization of institution','Region (IPEDS)','FIPS code for state','75th percentile of the ACT cumulative score','Midpoint of the ACT cumulative score','25th percentile of the ACT cumulative score','Average SAT equivalent score of students admitted for all campuses rolled up to the 6-digit OPE ID','75th percentile of SAT scores at the institution (writing)','Midpoint of SAT scores at the institution (writing)','25th percentile of SAT scores at the institution (writing)','75th percentile of SAT scores at the institution (math)','Midpoint of SAT scores at the institution (math)','25th percentile of SAT scores at the institution (math)','75th percentile of SAT scores at the institution (critical reading)','Midpoint of SAT scores at the institution (critical reading)','25th percentile of SAT scores at the institution (critical reading)','Admission rate for all campuses rolled up to the 6-digit OPE ID','Flag for women-only college','Flag for men-only college','Flag for Asian American Native American Pacific Islander-serving institution','Flag for predominantly black institution','Median earnings of students working and not enrolled 10 years after entry','Mean earnings of students working and not enrolled 10 years after entry','Median family income','Average family income','TUITIONFEE_PROG','Average faculty salary','Percentage of Pell Grant','The median debt for independent students','The median debt for students with family income between over 75k','The median debt for students with family income between $30k and 75k','The median debt for students with family income between $0 and 30k','The median debt for students who have not completed','The median debt for students who have completed','Five-year repayment rate by family income ($30k-75k)','Percent of students who never received a federal loan at the institution and who were still enrolled at original institution within 2 years','Percent of students who received a federal loan at the institution and who completed in 2 years ','Percent of students who did not receive a Pell Grant at the institution and who completed in 2 years at original ','Percent of students who received a Pell Grant at the institution and who completed in 2 years at original ','Percent of high-income (over in nominal family income) students who died within a year','Percent of middle-income (between $30k and 75k in nominal family income) students who died within a year','Percent of female students who transferred to a 2-year institution and whose status is unknown within 8 years','Percent died within 2 years at original institution','Percent of students who never received a federal loan at the institution and who were still enrolled at original institution within a year','Percent of students who never received a federal loan at the institution and who died within 2 years at original institution','Percent of students who received a federal loan at the institution and who died within 2 years at original institution','Percent of students who never received a Pell Grant at the institution and who died within 2 years at original institution','Percent of students who received a Pell Grant at the institution and who died within 2 years at original institution','Percent of high income (more than 75k in nominal family income) students who died within 2 years','Percent of middle-income (between $30k and 75k in nominal family income) students who died within 2 years','Percent of low income (between 0 and 30k in nominal family income) students who died within 2 years','The median debt for dependent students']
df = df.iloc[:,:]

In [99]:
DATA = pd.read_csv("Most+Recent+Cohorts+(All+Data+Elements).csv", delimiter=',', low_memory = False)
DATA = DATA.replace('PrivacySuppressed', '-1')
#Restricting the dataframe to 57 variables of interest
data =DATA.loc[:,['INSTNM','OPEID','UGDS','ADM_RATE_ALL','faminc','TUITIONFEE_PROG','DEP_DEBT_MDN']] 
data = data.set_index(['INSTNM'])
#Renaming the variables names by their definition
data.columns = ['8-digit OPE ID for institution','Enrollment of undergraduate degree-seeking students',
                'Admission rate for all campuses rolled up to the 6-digit OPE ID', 'Average family income',
                'TUITIONFEE_PROG','The median debt for dependent students']
data = data.iloc[:,:]

In [71]:
bdb = bayeslite.bayesdb_open("sim.bdb")

In [72]:
bdb = bayeslite.bayesdb_open("similarities.bdb")

In [68]:
bayesdb_read_pandas_df(bdb, "df", data, create=True)

In [63]:
# Link your dataframe to your bdb file
bayesdb_read_pandas_df(bdb, "simi", df, create=True)

In [73]:
# Load the education dataset into a local instance of bayeslite
ed = quickstart(name='df', bdb_path='sim.bdb')
q = ed.q

BQL [SELECT COUNT(*) FROM "df"] [()]
BQL [
        CREATE GENERATOR "df_cc" IF NOT EXISTS FOR "df" USING crosscat( GUESS(*) )] [()]


In [64]:
# Load the education dataset into a local instance of bayeslite
ed = quickstart(name='simi', bdb_path='similarities.bdb')
q = ed.q

BQL [SELECT COUNT(*) FROM "simi"] [()]
BQL [
        CREATE GENERATOR "simi_cc" IF NOT EXISTS FOR "simi" USING crosscat( GUESS(*) )] [()]


In [74]:
ed.analysis_status()

BQL [SELECT iterations FROM bayesdb_generator_model
                           WHERE generator_id = (
                            SELECT id FROM bayesdb_generator WHERE name = ?)] [('df_cc',)]


,count of models
iterations,


In [75]:
ed.analyze(models = 4, iterations=10)

BQL [INITIALIZE 4 MODELS IF NOT EXISTS FOR df_cc] [()]
BQL [ANALYZE df_cc FOR 10 ITERATIONS CHECKPOINT 1 ITERATION WAIT] [()]
BQL [SELECT iterations FROM bayesdb_generator_model
                           WHERE generator_id = (
                            SELECT id FROM bayesdb_generator WHERE name = ?)] [('df_cc',)]


,count of models
iterations,
10,4


Similarity of all the schools with respect to size

In [51]:
q('''
ESTIMATE SIMILARITY WITH RESPECT TO "Enrollment of undergraduate degree-seeking students" FROM PAIRWISE %g
''')

ERROR: 
Traceback (most recent call last):

  File "/Applications/Bayeslite-v0.1.7.app/Contents/MacOS/venv/lib/python2.7/site-packages/bdbcontrib/recipes.py", line 153, in query
    df = bdbcontrib.cursor_to_df(res)

  File "/Applications/Bayeslite-v0.1.7.app/Contents/MacOS/venv/lib/python2.7/site-packages/bdbcontrib/bql_utils.py", line 104, in cursor_to_df
    df = pd.DataFrame.from_records(cursor, coerce_float=True)

  File "/Applications/Bayeslite-v0.1.7.app/Contents/MacOS/venv/lib/python2.7/site-packages/pandas/core/frame.py", line 938, in from_records
    values += data

  File "/Applications/Bayeslite-v0.1.7.app/Contents/MacOS/venv/lib/python2.7/site-packages/bayeslite/bql.py", line 678, in next
    return self._cursor.next()

  File "src/connection.c", line 2021, in user-defined-scalar-bql_row_similarity

  File "/Applications/Bayeslite-v0.1.7.app/Contents/MacOS/venv/lib/python2.7/site-packages/bayeslite/bqlfn.py", line 34, in <lambda>
    db.createscalarfunction(name, (lambda *

BQL [
ESTIMATE SIMILARITY WITH RESPECT TO "Enrollment of undergraduate degree-seeking students" FROM PAIRWISE "sim_cc"
] [()]


An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



KeyboardInterrupt: 

In [85]:
q('''
select * from df where ("8-digit OPE ID for institution" = "00215500") ''')

BQL [
select * from df where ("8-digit OPE ID for institution" = "00215500") ] [()]


,8-digit OPE ID for institution,Enrollment of undergraduate degree-seeking students,Admission rate for all campuses rolled up to the 6-digit OPE ID,Average family income,TUITIONFEE_PROG,The median debt for dependent students
0,215500,7278,0.0584,60645.79,NaN,6500


In [ ]:
q('''
Estimate SIMILARITY WITH RESPECT TO (list of variable comma separated) FROM PAIRWISE ROWS OF df''')

Similarity of the schools to MIT with respect to several variables

"Enrollment of undergraduate degree-seeking students","Admission rate for all campuses rolled up to the 6-digit OPE ID", "Average family income", "The median debt for dependent students"


In [82]:
q(''' 
ESTIMATE SIMILARITY TO ("8-digit OPE ID for institution" = "00217800") WITH RESPECT TO 
("Enrollment of undergraduate degree-seeking students",
                "Admission rate for all campuses rolled up to the 6-digit OPE ID", "Average family income",
                "TUITIONFEE_PROG","The median debt for dependent students")
FROM %g''')

BQL [
ESTIMATE SIMILARITY TO ("8-digit OPE ID for institution" = "00217800") WITH RESPECT TO 
("Enrollment of undergraduate degree-seeking students",
                "Admission rate for all campuses rolled up to the 6-digit OPE ID", "Average family income",
                "TUITIONFEE_PROG","The median debt for dependent students")
FROM "df_cc"] [()]


,"bql_row_similarity(1, NULL, _rowid_, (SELECT _rowid_ FROM ""df"" WHERE (""8-digit OPE ID for institution"" = ""00217800"")), 1, 2, 3, 4, 5)"
0,0.25
1,0.30
2,0.00
3,0.20
4,0.20
5,0.00
6,0.00
7,0.00
8,0.20
9,0.20


Similarity between MIT and Harvard with respect to several variables

In [86]:
q('''
ESTIMATE SIMILARITY TO ("8-digit OPE ID for institution" = "00217800")
WITH RESPECT TO
("Enrollment of undergraduate degree-seeking students",
                "Admission rate for all campuses rolled up to the 6-digit OPE ID", "Average family income",
                "TUITIONFEE_PROG","The median debt for dependent students")
FROM %g WHERE ("8-digit OPE ID for institution" = "00215500")''')

BQL [
ESTIMATE SIMILARITY TO ("8-digit OPE ID for institution" = "00217800")
WITH RESPECT TO
("Enrollment of undergraduate degree-seeking students",
                "Admission rate for all campuses rolled up to the 6-digit OPE ID", "Average family income",
                "TUITIONFEE_PROG","The median debt for dependent students")
FROM "df_cc" WHERE ("8-digit OPE ID for institution" = "00215500")] [()]


,"bql_row_similarity(1, NULL, _rowid_, (SELECT _rowid_ FROM ""df"" WHERE (""8-digit OPE ID for institution"" = ""00217800"")), 1, 2, 3, 4, 5)"
0,0.4


In [91]:
q('''
ESTIMATE SIMILARITY TO ("8-digit OPE ID for institution" = "00217800")
WITH RESPECT TO "TUITIONFEE_PROG"
FROM %g WHERE ("8-digit OPE ID for institution" = "00215500")''')

BQL [
ESTIMATE SIMILARITY TO ("8-digit OPE ID for institution" = "00217800")
WITH RESPECT TO "TUITIONFEE_PROG"
FROM "df_cc" WHERE ("8-digit OPE ID for institution" = "00215500")] [()]


,"bql_row_similarity(1, NULL, _rowid_, (SELECT _rowid_ FROM ""df"" WHERE (""8-digit OPE ID for institution"" = ""00217800"")), 4)"
0,0


In [92]:
q('''
ESTIMATE SIMILARITY TO ("8-digit OPE ID for institution" = "00217800")
WITH RESPECT TO "Admission rate for all campuses rolled up to the 6-digit OPE ID"
FROM %g WHERE ("8-digit OPE ID for institution" = "00215500")''')

BQL [
ESTIMATE SIMILARITY TO ("8-digit OPE ID for institution" = "00217800")
WITH RESPECT TO "Admission rate for all campuses rolled up to the 6-digit OPE ID"
FROM "df_cc" WHERE ("8-digit OPE ID for institution" = "00215500")] [()]


,"bql_row_similarity(1, NULL, _rowid_, (SELECT _rowid_ FROM ""df"" WHERE (""8-digit OPE ID for institution"" = ""00217800"")), 2)"
0,0.5


In [102]:
DATA = pd.read_csv("Most+Recent+Cohorts+(All+Data+Elements).csv", delimiter=',', low_memory = False)

In [103]:
DATA

,UNITID,OPEID,opeid6,INSTNM,CITY,STABBR,ZIP,AccredAgency,INSTURL,NPCURL,...,PELL_YR8_N,NOPELL_YR8_N,LOAN_YR8_N,NOLOAN_YR8_N,FIRSTGEN_YR8_N,NOT1STGEN_YR8_N,REPAY_DT_MDN,SEPAR_DT_MDN,REPAY_DT_N,SEPAR_DT_N
0,100654,100200,1002,Alabama A & M University,Normal,AL,35762,Southern Association of Colleges and Schools C...,www.aamu.edu/,galileo.aamu.edu/netpricecalculator/npcalc.htm,...,1089,247,1225,111,485,657,379852,379852,1788,2182
1,100663,105200,1052,University of Alabama at Birmingham,Birmingham,AL,35294-0110,Southern Association of Colleges and Schools C...,www.uab.edu,www.collegeportraits.org/AL/UAB/estimator/agree,...,1440,1010,2236,214,847,1365,379852,379852,3681,4495
2,100690,2503400,25034,Amridge University,Montgomery,AL,36117-3553,Southern Association of Colleges and Schools C...,www.amridgeuniversity.edu,tcc.noellevitz.com/(S(miwoihs5stz5cpyifh4nczu0...,...,99,46,PrivacySuppressed,PrivacySuppressed,70,57,19524,19467,266,406
3,100706,105500,1055,University of Alabama in Huntsville,Huntsville,AL,35899,Southern Association of Colleges and Schools C...,www.uah.edu,finaid.uah.edu/,...,626,392,934,84,303,619,379852,379852,1607,1934
4,100724,100500,1005,Alabama State University,Montgomery,AL,36104-0271,Southern Association of Colleges and Schools C...,www.alasu.edu/email/index.aspx,www.alasu.edu/cost-aid/forms/calculator/index....,...,1290,208,1394,104,565,678,379852,379852,2579,3176
5,100751,105100,1051,The University of Alabama,Tuscaloosa,AL,35487-0166,Southern Association of Colleges and Schools C...,www.ua.edu/,oira.ua.edu,...,1515,1238,2584,169,762,1816,379852,379852,7476,8294
6,100760,100700,1007,Central Alabama Community College,Alexander City,AL,35010,Southern Association of Colleges and Schools C...,www.cacc.edu,www.cacc.edu/graphics/netpricecalculator/npcal...,...,611,91,374,328,369,195,379852,19576,615,933
7,100812,100800,1008,Athens State University,Athens,AL,35611,Southern Association of Colleges and Schools C...,www.athens.edu,24.athens.edu/apex/prod8/f?p=174:1:39413574495...,...,487,231,627,91,368,282,379852,379852,1105,1498
8,100830,831000,8310,Auburn University at Montgomery,Montgomery,AL,36117-3596,Southern Association of Colleges and Schools C...,www.aum.edu,www.aum.edu/admissions/financial-information/p...,...,769,355,1017,107,418,556,379852,19569,1673,2075
9,100858,100900,1009,Auburn University,Auburn University,AL,36849,Southern Association of Colleges and Schools C...,www.auburn.edu,www.auburn.edu/admissions/money-matters.html,...,1186,1357,2430,113,566,1852,379852,379852,4230,4600


In [ ]:
sim_1_var = q('''
ESTIMATE "INSTNM", SIMILARITY WITH RESPECT TO ("TUITIONFEE_PROG") as value
FROM PAIRWISE %g
ORDER BY value DESC 
''')

In [ ]:
sim_mit = q(''' 
ESTIMATE "INSTNM", SIMILARITY TO ("key" = "00217800") WITH RESPECT TO 
("Enrollment of undergraduate degree-seeking students",
                "Admission rate for all campuses rolled up to the 6-digit OPE ID", "Average family income",
                "TUITIONFEE_PROG","The median debt for dependent students") as value
FROM %g
ORDER BY value DESC ''')